In [10]:

from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import label_binarize
from itertools import cycle

import warnings
import os
import time
import math
import requests

import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')

In [11]:
df = pd.read_csv('EPL_Data_featured.csv')

In [21]:
df.head()

,week,date,home_team,home_xg,score,away_xg,away_team,referee,game_id,home_team_elo,...,home_goals_conceded_to_date,away_goals_conceded_to_date,home_match_points,away_match_points,home_points_to_date,away_points_to_date,match_points,home_form,away_form,match_result
0,1,2017-08-11,arsenal,2.5,4-3,1.5,leicester city,Mike Dean,e3c3ddf0,1848.286499,...,0,0,3,0,0,0,3,0.0,0.0,0
1,1,2017-08-12,brighton,0.3,0-2,1.9,manchester city,Michael Oliver,072bfc99,1583.799805,...,0,0,0,3,0,0,3,0.0,0.0,2
2,1,2017-08-12,chelsea,1.5,2-3,0.6,burnley,Craig Pawson,71b00bca,1909.399658,...,0,0,0,3,0,0,3,0.0,0.0,2
3,1,2017-08-12,crystal palace,1.1,0-3,1.5,huddersfield,Jonathan Moss,2d369d17,1642.862427,...,0,0,0,3,0,0,3,0.0,0.0,2
4,1,2017-08-12,everton,0.6,1-0,0.4,stoke city,Niel Swarbrick,7c834541,1751.501343,...,0,0,3,0,0,0,3,0.0,0.0,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   week                         2280 non-null   int64  
 1   date                         2280 non-null   object 
 2   home_team                    2280 non-null   object 
 3   home_xg                      2280 non-null   float64
 4   score                        2280 non-null   object 
 5   away_xg                      2280 non-null   float64
 6   away_team                    2280 non-null   object 
 7   referee                      2280 non-null   object 
 8   game_id                      2280 non-null   object 
 9   home_team_elo                2280 non-null   float64
 10  away_team_elo                2280 non-null   float64
 11  season                       2280 non-null   int64  
 12  home_starters                2280 non-null   object 
 13  away_starters     

In [12]:
df.columns

Index(['week', 'date', 'home_team', 'home_xg', 'score', 'away_xg', 'away_team',
       'referee', 'game_id', 'home_team_elo', 'away_team_elo', 'season',
       'home_starters', 'away_starters', 'home_xG_to_date', 'away_xG_to_date',
       'home_xG_against_to_date', 'away_xG_against_to_date',
       'home_goals_scored', 'away_goals_scored', 'home_goals_scored_to_date',
       'away_goals_scored_to_date', 'home_goals_conceded_to_date',
       'away_goals_conceded_to_date', 'home_match_points', 'away_match_points',
       'home_points_to_date', 'away_points_to_date', 'match_points',
       'home_form', 'away_form', 'match_result'],
      dtype='object')

In [13]:
columns_to_drop = ['week', 'date', 'home_team','home_xg', 'score', 'away_xg','away_team','game_id',
                   'season','home_match_points', 'away_match_points', 'match_points',
                   'home_goals_scored', 'away_goals_scored', 'home_starters', 'away_starters']

In [14]:
df_dropped = df.drop(columns=columns_to_drop)

# Display the first few rows of the dataframe after dropping the columns
df_dropped.head()

,referee,home_team_elo,away_team_elo,home_xG_to_date,away_xG_to_date,home_xG_against_to_date,away_xG_against_to_date,home_goals_scored_to_date,away_goals_scored_to_date,home_goals_conceded_to_date,away_goals_conceded_to_date,home_points_to_date,away_points_to_date,home_form,away_form,match_result
0,Mike Dean,1848.286499,1716.994873,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0
1,Michael Oliver,1583.799805,1866.807007,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2
2,Craig Pawson,1909.399658,1628.988403,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2
3,Jonathan Moss,1642.862427,1475.799316,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,2
4,Niel Swarbrick,1751.501343,1662.613770,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0


In [15]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   referee                      2280 non-null   object 
 1   home_team_elo                2280 non-null   float64
 2   away_team_elo                2280 non-null   float64
 3   home_xG_to_date              2280 non-null   float64
 4   away_xG_to_date              2280 non-null   float64
 5   home_xG_against_to_date      2280 non-null   float64
 6   away_xG_against_to_date      2280 non-null   float64
 7   home_goals_scored_to_date    2280 non-null   int64  
 8   away_goals_scored_to_date    2280 non-null   int64  
 9   home_goals_conceded_to_date  2280 non-null   int64  
 10  away_goals_conceded_to_date  2280 non-null   int64  
 11  home_points_to_date          2280 non-null   int64  
 12  away_points_to_date          2280 non-null   int64  
 13  home_form         

In [16]:

# Assuming df_dropped is your DataFrame after dropping columns

# Define the target variable and the feature set
X = df_dropped.drop('match_result', axis=1)
y = df_dropped['match_result']

# Identify categorical features for CatBoost
categorical_features_indices = list(X.select_dtypes(include=['object']).columns)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CatBoostClassifier
catboost_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=3,
    cat_features=categorical_features_indices,
    verbose=10
)

# Train CatBoostClassifier
catboost_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = catboost_model.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' if classes are imbalanced
print(f'The F1 score of the CatBoost model is: {f1}')


0:	learn: 1.0767104	total: 4.42ms	remaining: 438ms
10:	learn: 0.9919540	total: 21.6ms	remaining: 175ms
20:	learn: 0.9699701	total: 35.9ms	remaining: 135ms
30:	learn: 0.9547843	total: 47.9ms	remaining: 107ms
40:	learn: 0.9473945	total: 61.4ms	remaining: 88.3ms
50:	learn: 0.9398132	total: 77.3ms	remaining: 74.3ms
60:	learn: 0.9330621	total: 91.5ms	remaining: 58.5ms
70:	learn: 0.9276081	total: 104ms	remaining: 42.4ms
80:	learn: 0.9230261	total: 115ms	remaining: 27ms
90:	learn: 0.9180101	total: 126ms	remaining: 12.4ms
99:	learn: 0.9130461	total: 136ms	remaining: 0us
The F1 score of the CatBoost model is: 0.46371920437921116


In [17]:



# Assuming X and y are already defined and categorical features are properly handled

# Create a CatBoost classifier instance
catboost = CatBoostClassifier(cat_features=categorical_features_indices, verbose=0)

# Define the pipeline
pipeline = Pipeline([
    # Here you can add other preprocessing steps if needed
    ('classifier', catboost)
])

# Define the hyperparameters grid to be tested
params_grid = {
    'classifier__iterations': [25, 50, 100],
    'classifier__depth': [5, 6, 7, 8],
    'classifier__learning_rate': [ 0.1, 0.2, 0.3, 0.4]
    # You can add other parameters you want to tune
}

# Setup the GridSearchCV object
grid_search = GridSearchCV(pipeline, params_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best hyperparameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Predict on the testing set using the best estimator
y_pred = best_estimator.predict(X_test)

# Calculate the F1 score with the best estimator
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Best parameters: {best_params}')
print(f'Best F1 score: {f1}')


Best parameters: {'classifier__depth': 6, 'classifier__iterations': 50, 'classifier__learning_rate': 0.4}
Best F1 score: 0.4804435796908426


In [18]:
# Get feature importances from the best estimator
feature_importances = best_estimator.named_steps['classifier'].get_feature_importance()

# Get feature names
feature_names = X_train.columns

# Create a DataFrame to display feature importances
importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort the DataFrame to show the most important features at the top
importances_df.sort_values(by='Importance', ascending=False, inplace=True)

print(importances_df)


                        Feature  Importance
1                 home_team_elo   14.828051
0                       referee   14.333667
2                 away_team_elo   13.144391
13                    home_form    8.760014
6       away_xG_against_to_date    6.205130
14                    away_form    6.003362
8     away_goals_scored_to_date    5.715681
12          away_points_to_date    5.060634
5       home_xG_against_to_date    4.933198
3               home_xG_to_date    4.140532
10  away_goals_conceded_to_date    3.803018
4               away_xG_to_date    3.582775
11          home_points_to_date    3.558526
7     home_goals_scored_to_date    3.327620
9   home_goals_conceded_to_date    2.603401
